In [17]:
import os
api_key = os.environ['OpenAIKey']



In [70]:
import glob
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_core.documents import Document

# Path to all .md files in the folder
markdown_path = "CollegeInfo/allinfo.md"
loader = UnstructuredMarkdownLoader(markdown_path)

data = loader.load()
assert len(data) == 1
assert isinstance(data[0], Document)
readme_content = data[0].page_content
print(readme_content[:250])

#College Info for Engineering Colleges

College 1: BMS College of Engineering

BMS College Overview Duration 4 years

Course Level UG Degree

Mode of Course Full Time

Total Tuition Fees INR 18.00 Lakh

Type of Univers ity Private,Autonomous

BMS Col


In [72]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
splits = text_splitter.split_documents(data)

In [73]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

# Load a local embedding model from Hugging Face
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create your vectorstore
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)


In [74]:
print(vectorstore._collection.count())

877


In [75]:
retriever = vectorstore.as_retriever()

In [76]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

c:\Users\rohan\miniconda3\envs\LangchainRAG\lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [77]:
from dotenv import load_dotenv
load_dotenv()

True

In [78]:
from langchain_core.runnables import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [79]:
def format_docs(docs):
    return "\n".join(doc.page_content for doc in docs)


In [80]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    model="llama3-70b-8192",  # or use "llama3-70b-8192"
    openai_api_key="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",
    openai_api_base="https://api.groq.com/openai/v1"
)


In [82]:
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

retriever =vectorstore.as_retriever()

rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)


In [85]:
response = rag_chain.invoke({"query": "Recommend college  for a student interested in Mechanical Engineering and has a general rank 0f 1500, include the cutoff and placement details"})
print(response['result'])


Based on the provided context, I recommend the Department of Mechanical Engineering in the institution mentioned. Here's why:

* The department has a strong reputation, being one of the oldest departments in the institution, with a rich history of 40 years.
* The department offers a four-year bachelor's degree program in Mechanical Engineering, which is a great fit for the student's interest.
* The faculty members are well-qualified, with 38 members holding a minimum postgraduate degree, and 26 of them possessing a doctoral degree.
* The department has a strong research focus, being a VTU-recognized R&D Centre for MSc Engineering and Ph.D. programs.
* The institution has a Full-fledged Placement Centre that is active and successful.

Regarding cutoff and placement details, I don't have specific information for the Mechanical Engineering department. However, I can provide some general insights based on the context:

* The institution has a general rank of 1500, which is a decent rank. F